In [ ]:
# Importar bibliotecas utilizadas
import pandas as pd

import numpy as np

import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer

# Regular expression
import re

from wordcloud import WordCloud

import matplotlib.pyplot as plt


In [ ]:
# Set of predefined words NLTK (Natural Language Toolkit)
import nltk 

#nltk.download('all')
set(stopwords.words('english'))

from nltk.tokenize import word_tokenize 

from nltk.corpus import stopwords 
#from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

print(stopwords.words('english'))

In [ ]:
# Coleta dados
catalogoVulnerabilidades = pd.read_csv('dados/known_exploited_vulnerabilities.csv', header=0, sep=',')

# Criar coluna booleana state com 0 para Unknown e 1 para Known
catalogoVulnerabilidades['codigoKnownRansomwareCampaignUse'] = np.where(catalogoVulnerabilidades['knownRansomwareCampaignUse'] == 'Known', 1, 0 )

#Reordenando as colunas
catalogoVulnerabilidades = catalogoVulnerabilidades[['cveID', 
                                                     'vendorProject',
                                                     'product',
                                                     'vulnerabilityName',
                                                     'dateAdded',
                                                     'shortDescription',
                                                     'requiredAction',
                                                     'dueDate',
                                                     'knownRansomwareCampaignUse',
                                                     'codigoKnownRansomwareCampaignUse',
                                                     'notes']]    



In [ ]:
# Verificar tipos de colunas
catalogoVulnerabilidades.info()

In [ ]:
# Excluir a coluna notes
catalogoVulnerabilidades.drop(columns=['notes'], inplace = True)

In [ ]:
# Verificar alguns dados
catalogoVulnerabilidades.head()

In [ ]:
# Transformar tipos de dados
catalogoVulnerabilidades.dateAdded = pd.to_datetime(catalogoVulnerabilidades.dateAdded)
catalogoVulnerabilidades.dueDate = pd.to_datetime(catalogoVulnerabilidades.dueDate)
##Tranforma a coluna codigoKnownRansomwareCampaignUse para boolean
catalogoVulnerabilidades['codigoKnownRansomwareCampaignUse'] = catalogoVulnerabilidades['codigoKnownRansomwareCampaignUse'].astype('bool')
##Transforma colunas em string
catalogoVulnerabilidades['shortDescription'] = catalogoVulnerabilidades['shortDescription'].astype('string')

In [ ]:
# Verificar tipos de colunas após transformação
catalogoVulnerabilidades.info()

In [ ]:
# Pareto antes de limpar coluna

In [ ]:
# Função para limpar os textos das colunas do dataframe
def limpa_coluna(corpus):
    return re.sub(r'[^a-zA-Z0-9\s]', '', corpus)

In [ ]:
# Limpa a coluna 
pd_catalogoVulnerabilidades = pd.DataFrame(catalogoVulnerabilidades)
pd_catalogoVulnerabilidades['shortDescription'] = pd_catalogoVulnerabilidades['shortDescription'].apply(lambda x: limpa_coluna(str(x).lower()))

pd_catalogoVulnerabilidades.shortDescription

In [ ]:
# Função de Tokenização
def tokenizacao (corpus):
    tokens = word_tokenize(corpus)
    return tokens

In [ ]:
# Copia todos os registros
pd_catalogoVulnerabilidades_tk['shortDescription'] = pd_catalogoVulnerabilidades['shortDescription'].copy()
pd_catalogoVulnerabilidades_tk['shortDescription'] = pd_catalogoVulnerabilidades_tk['shortDescription'].apply(lambda x: tokenizacao(str(x)))

pd_catalogoVulnerabilidades_tk.shortDescription.head()

In [ ]:
# Realiza a tokenização
tokens_contagem = ' '.join([word for word in pd_catalogoVulnerabilidades.shortDescription])

tokens_contagem = tokenizacao(tokens_contagem)

tokens_frequencia = nltk.FreqDist(tokens_contagem)

df_frequencia_tk = pd.DataFrame({"token": list(tokens_frequencia.keys()),
                                 "frequencia": list(tokens_frequencia.values())})


df_frequencia_tk.nlargest(columns = "frequencia", n = 10)




In [ ]:
# Gráfico de Pareto antes de remover stop words
plt.figure(figsize=(10, 7))
x = sns.barplot(data = df_frequencia_tk.nlargest(columns = "frequencia", n = 10), x = "token", y = "frequencia", color = 'blue')
x.set(ylabel = "Quantidade", xlabel = "Tokens", title = 'Contendo stop words')
plt.show()


In [ ]:
# Função remover stop words
def remove_stop_words(word_list, stop_word_list):
    lista_sem_stop_words = [word for word in word_list 
                            if word.lower() not in stop_word_list]
    return lista_sem_stop_words

In [ ]:
# Carrega lista de stop words do inglês
sw_en = set(stopwords.words('english'))

tokens_sem_stop_words = remove_stop_words(tokens_contagem, sw_en)

In [ ]:
# Cria DataFrame sem stop words

tokens_sem_stop_words_frequencia = nltk.FreqDist(tokens_sem_stop_words)

df_sem_stop_words_frequencia = pd.DataFrame({"token": list(tokens_sem_stop_words_frequencia.keys()),
                                 "frequencia": list(tokens_sem_stop_words_frequencia.values())})


df_sem_stop_words_frequencia.nlargest(columns = "frequencia", n = 10)

In [ ]:
# Gráfico de Pareto depois de remover stop words
plt.figure(figsize=(10, 7))
x = sns.barplot(data = df_sem_stop_words_frequencia.nlargest(columns = "frequencia", n = 10), x = "token", y = "frequencia", color = 'blue')
x.set(ylabel = "Quantidade", xlabel = "Tokens", title = 'Sem stop words')
plt.show()


In [ ]:
# Função de Lematização
def lematizacao (df, nome_coluna):
    df[nome_coluna] = df[nome_coluna].map(lambda x: WordNetLemmatizer.lemmatize(x))
    return df

In [ ]:
# Vetorizar coluna
vetorizar = CountVectorizer(lowercase = False, max_features=100)
bow = vetorizar.fit_transform(catalogoVulnerabilidades.shortDescription)

print(bow.shape)

In [ ]:
# Tranforma matriz esparsa em Dataframe
df_shortDescription_bow = pd.DataFrame.sparse.from_spmatrix(bow, columns=vetorizar.get_feature_names_out())

df_shortDescription_bow.head()

In [ ]:
# Wordcloud
shortDescription_all_words = ' '.join([word for word in pd_catalogoVulnerabilidades.shortDescription])

# Quantidade de palavras
print(len(shortDescription_all_words))

In [ ]:
# Gerar wordcloud

## https://amueller.github.io/word_cloud/generated/wordcloud.WordCloud.html

shortDescription_wc = WordCloud(width= 800, height= 500, max_font_size = 110, collocations=False).generate(shortDescription_all_words)

In [ ]:
# Plotar wordcloud
plt.figure(figsize=(12, 8.5))
plt.imshow(shortDescription_wc, interpolation='bilinear') #ver outras interpolações
plt.axis("off")
plt.show()